In [129]:
cd S:\Datasets

S:\Datasets


In [130]:
from pathlib import Path
from datetime import datetime
import csv
import pandas as pd
import numpy as np

base_path = Path.cwd().parent / 'Datasets' / 'Solar' / 'daily' / 'M18'
model_path = Path.cwd().parent / 'Datasets' / 'Solar'
print(base_path)

S:\Datasets\Solar\daily\M18


In [131]:
import pandas as pd
import numpy as np

grid = pd.concat([pd.read_csv(base_path/"MACA_inputs.csv", header=0)])

print(len(grid))

grid.head()



782


,Name
0,S:\Datasets\Solar\daily\M18\GCM_Projections_ce...
1,S:\Datasets\Solar\daily\M18\GCM_Projections_ce...
2,S:\Datasets\Solar\daily\M18\GCM_Projections_ce...
3,S:\Datasets\Solar\daily\M18\GCM_Projections_ce...
4,S:\Datasets\Solar\daily\M18\GCM_Projections_ce...


In [132]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
import time

NN = tf.keras.models.load_model(model_path / "Solar V2.0.h5")

open(base_path / 'CONUS_Predictions_Final.csv', 'w')


<_io.TextIOWrapper name='S:\\Datasets\\Solar\\daily\\M18\\CONUS_Predictions_Final.csv' mode='w' encoding='cp1252'>

In [133]:
column_list = ["Unnamed: 0", 
               "Month",
               #"Day",
               #"MW", 
               #"Min_Daily_Relative_Humidity",
               #"Max_Daily_Relative_Humidity",
               #"Downwelling_Solar_Radiation", 
               #"Daily_Minimum_Temperature", 
               #"Daily_Maximum_Temperature", 
               #"Daily_Eastward_Wind_Component", 
               #"Daily_Northward_Wind_Component",
               #"Precipitation",
               #"Specific_Humidity"
               "Avg_Daily_GHI",
               "Latitude", 
               "Longitude", 
               "Daily_Average_Power", 
               "Monthly_Average_Power",
               #"Yearly_Average_Power",
               "Cell",
               #"Elevation",
               #"Solar_Zenith_Angle",
              ]

In [134]:
i = 9

file_path = grid['Name'][i]

data_input = pd.concat([pd.read_csv(file_path, header=0)])

data_input.head()

,Unnamed: 0,Month,MW,rhsmin,rhsmax,rsds,tasmin,tasmax,uas,vas,pr,huss,Latitude,Longitude
0,0,1,37,16.644660,58.440228,124.663055,1.698181,17.523651,-1.297266,1.905906,0.0,0.002533,29.0,-100.0
1,1,1,37,19.591139,66.978320,75.145580,5.016052,18.091797,-1.953203,1.995286,0.0,0.003803,29.0,-100.0
2,2,1,37,6.451661,73.684540,150.700180,9.578308,27.441590,1.425707,-1.884599,0.0,0.004357,29.0,-100.0
3,3,1,37,5.305518,28.732038,151.797970,5.307739,24.510742,-1.090099,-1.887258,0.0,0.001798,29.0,-100.0
4,4,1,37,6.251544,27.494781,152.684780,5.961700,28.089050,-0.923557,2.995532,0.0,0.001746,29.0,-100.0


In [135]:
for i in range(1, len(grid)):
    
    try:

        # Import the ith document listed in grid

        file_path = grid['Name'][i]

        data_input = pd.concat([pd.read_csv(file_path, header=0)])

        data_input = data_input.dropna().reset_index(drop=True)

        # Arranging columns to match expected order
        cols = data_input.columns.tolist()
        cols_order = [0, 1, 12, 13, 3, 4, 5, 6, 7, 8, 9, 10, 11, 2]
        cols = [cols[i] for i in cols_order]
        data_input = data_input[cols]
        
        data_input = data_input.rename(columns={'rhsmin': 'Min_Daily_Relative_Humidity', 'rhsmax': 'Max_Daily_Relative_Humidity',
                                               'tasmin': 'Daily_Minimum_Temperature', 'tasmax': 'Daily_Maximum_Temperature',
                                                'uas': 'Daily_Eastward_Wind_Component', 'vas': 'Daily_Northward_Wind_Component', 
                                                'rsds': 'Avg_Daily_GHI'})

        data_input['Pivot'] = "DPV"

        # Drop columns from input that aren't needed
        #imputer = SimpleImputer(strategy="mean")

        # Selection of inputs for training in preprocessing pipeline
        data_input = data_input.drop([#"Unnamed: 0",
                                          "pr",
                                          "huss",
                                          "MW",
                                          #"Pivot",
                                          #"uas",
                                          #"vas"
                                          ], axis = 1)
        #imputer.fit(data_input_num)

        # Imputing and normalizing for the selected features

        import joblib
        scaler_filename = "Solar_Pipeline_Scaler.save"
        full_pipeline = joblib.load(model_path / scaler_filename)

        input_prepared = full_pipeline.transform(data_input)

        input_predictions = NN.predict(input_prepared)

        data_input['Daily_Average_Power'] = input_predictions

        data_input['Cell'] = i

        # Making monthly averages
        shifted_x = data_input['Daily_Average_Power'].shift(1)
        window_x = shifted_x.rolling(window = 30, center = False)

        data_input['Monthly_Average_Power'] = window_x.mean()

        data_input['New_Month'] = data_input['Month'] - data_input['Month'].shift(-1)

        data_input = data_input[data_input.New_Month != 0]
        data_input = data_input[data_input.New_Month != 1]
        
        #print(i)


        if i == 1:
            # Append the values to our daily data file - the first one provides the header
            data_input.to_csv(base_path / 'CONUS_Predictions_Final.csv', mode='a', index=False, header=True, columns = column_list)
            print(((i/(len(grid)))*100), "%", end='\r')

        else:
            data_input.to_csv(base_path / 'CONUS_Predictions_Final.csv', mode='a', index=False, header=False, columns = column_list)
            print(((i/(len(grid)))*100), "%", end='\r')

    #except Exception as e: print(e)
    except:

        print("ooops", end = '\r')
        #print(exception)

In [136]:
import winsound

#This was a triumph
winsound.Beep(392, 250)
winsound.Beep(370, 250)
winsound.Beep(330, 250)
winsound.Beep(330, 250)
winsound.Beep(370, 1000)

#I'm making a note here, huge success
winsound.Beep(220, 250)
winsound.Beep(392, 250)
winsound.Beep(370, 250)
winsound.Beep(330, 500)
winsound.Beep(330, 750)
winsound.Beep(370, 500)
winsound.Beep(294, 750)
winsound.Beep(330, 500)
winsound.Beep(220, 1250)

#It's hard to overstate my satisfaction
winsound.Beep(220, 250)
winsound.Beep(330, 500)
winsound.Beep(370, 250)
winsound.Beep(392, 750)
winsound.Beep(330, 250)
winsound.Beep(277, 500)
winsound.Beep(293, 250)
winsound.Beep(330, 750)
winsound.Beep(220, 250)
winsound.Beep(220, 500)
winsound.Beep(370, 750)